# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [7]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [8]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# UnderSampling

In [9]:
from imblearn.under_sampling import RandomUnderSampler

X_data = mbti['posts']
X = X_data.values.reshape(-1, 1)

y_data = mbti['type']
print(y_data)
y = y_data.values.reshape(-1, 1)
print(y)

X_resampled, y_resampled = RandomUnderSampler(random_state=0).fit_sample(X, y)
#format(Counter(y_resampled))
X_resampled #ndarray
Counter(y_resampled)


0       ENFJ
1       ENFJ
2       ENFJ
3       ENFJ
4       ENFJ
        ... 
8670    ISTP
8671    ISTP
8672    ISTP
8673    ISTP
8674    ISTP
Name: type, Length: 8675, dtype: object
[['ENFJ']
 ['ENFJ']
 ['ENFJ']
 ...
 ['ISTP']
 ['ISTP']
 ['ISTP']]


Counter({'ENFJ': 39,
         'ENFP': 39,
         'ENTJ': 39,
         'ENTP': 39,
         'ESFJ': 39,
         'ESFP': 39,
         'ESTJ': 39,
         'ESTP': 39,
         'INFJ': 39,
         'INFP': 39,
         'INTJ': 39,
         'INTP': 39,
         'ISFJ': 39,
         'ISFP': 39,
         'ISTJ': 39,
         'ISTP': 39})

# tokenize and stemming

In [10]:
X_data = X_resampled.ravel()
y_data = y_resampled


print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    rm_urls = re.sub(r'http[s]?://\S+', '', sentence)
    clean_sentence = re.sub('[^A-Za-z\s]+', '', rm_urls.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [11]:
import nltk
import gc
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [12]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stop_words = set(stopwords.words('english'))
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        if word not in stop_words:
            tempData.append(ps.stem(word))
    stemData.append(tempData)
print("stemming 완료")
print(stemData[0])
stemData = np.array(stemData)
#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]
del [[mbti]]
gc.collect()

stemming 시작
stemming 완료
['want', 'know', 'feel', 'intp', 'actual', 'intp', 'idea', 'intp', 'havent', 'met', 'anyten', 'million', 'hug', 'mani', 'amaz', 'convers', 'held', 'absenc', 'vow', 'never', 'leav', 'realli', 'lot', 'written', 'recent', 'terriblyfacebook', 'way', 'emphas', 'other', 'error', 'cognit', 'becom', 'obviou', 'lead', 'wonder', 'im', 'friend', 'mani', 'peopl', 'rememb', 'peopl', 'moremmmmhm', 'ive', 'wonder', 'late', 'ei', 'think', 'definit', 'much', 'needwant', 'compani', 'around', 'regardless', 'time', 'togeth', 'spenthahah', 'love', 'think', 'that', 'go', 'topic', 'excel', 'convers', 'thank', 'particip', 'indulg', 'interest', 'enlighteningsomeon', 'creat', 'chitchat', 'thread', 'enfj', 'forum', 'enfj', 'vigilantli', 'await', 'type', 'wander', 'start', 'conversationswa', 'enfp', 'ive', 'caught', 'mani', 'text', 'look', 'around', 'shini', 'object', 'give', 'midsyl', 'mm', 'hmm', 'alway', 'mani', 'andi', 'tire', 'extravert', 'energysap', 'sentiment', 'arent', 'batshit', 

0

# Vectorization

In [13]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
print(tfidf.transform(stemData).toarray())
X = tfidf.transform(stemData)
y = np.array(y_data)


[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.05858548 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.04542902 0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


# XGBoost_Undersampling

In [15]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

In [16]:
print("Model building Start")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)
model = XGBClassifier()
model.fit(X_train, y_train, verbose=1)
#model.get_booster().get_score(importance_type="gain") #변수의 중요도

Model building Start


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [17]:
import xgboost as xgb
#test_hp = xgb.DMatrix(data=VHX_data, label=hy)
#preds = model.predict(X_test)
preds = model.predict(X_test)
print("Model building End")

Model building End


In [18]:
import joblib

joblib.dump(model, 'XGBoost_undersample.pkl')

['XGBoost_undersample.pkl']

In [19]:
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

print("Evaluation Start")
# labels과 guesses
labels = preds
guesses = y_test

print(accuracy_score(labels, guesses))
print(recall_score(labels, guesses, average=None))
print(precision_score(labels, guesses, average=None))
print(f1_score(labels, guesses, average=None))

print("Evaluation END")
# https://eunsukimme.github.io/ml/2019/10/21/Accuracy-Recall-Precision-F1-score/

Evaluation Start
0.4946808510638298
[0.71428571 0.4        0.27272727 0.23529412 0.81818182 0.41666667
 0.54545455 1.         0.33333333 0.7        0.375      0.16666667
 0.5        0.46666667 0.61111111 0.58333333]
[0.625      0.30769231 0.375      0.4        0.5625     0.45454545
 0.4        0.61538462 0.25       0.53846154 0.33333333 0.2
 0.66666667 0.7        0.61111111 0.63636364]
[0.66666667 0.34782609 0.31578947 0.2962963  0.66666667 0.43478261
 0.46153846 0.76190476 0.28571429 0.60869565 0.35294118 0.18181818
 0.57142857 0.56       0.61111111 0.60869565]
Evaluation END


In [20]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import cross_val_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
score = cross_val_score(model, X_train, y_train, cv=skf )
print(score)
print(score.mean())

[0.46590909 0.4137931  0.50574713 0.48275862 0.42528736]
0.45869905956112855


In [21]:
import nltk
import numpy as np
import pandas as pd

from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import re
import glob   

'''HP data Normalization and Stemming'''
# bag_of_words = {} #{등장인물 : 해당인물의 stemData, ~}
h_normalized_text = []
h_stemData = []

ps=PorterStemmer()
stop_words = set(stopwords.words('english'))
input_file = r'.\data\\test\\'

allHPfile_list = glob.glob(os.path.join(input_file, 'HP_*'))
print(allHPfile_list)

harryPotter={}
for file in allHPfile_list:
    df = pd.read_csv(file, sep=',', encoding='iso-8859-1')
    harryPotter[df['Character'][0]] = df['Sentence'].ravel()

print("stemming 시작")
for element in harryPotter.keys():
    h_normalized_text = []
    h_stemData = []
    for sentence in harryPotter[element]:
        h_clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
        h_normalized_text.append(h_clean_sentence)
    for sentence in h_normalized_text:
        tokenData = nltk.word_tokenize(sentence)
        for word in tokenData:
            if word not in stop_words:
                h_stemData.append(ps.stem(word))
    harryPotter[element] = h_stemData
    print(harryPotter[element])

print("stemming 완료")

['.\\data\\\\test\\HP_DUMBLEDORE.csv', '.\\data\\\\test\\HP_HAGRID.csv', '.\\data\\\\test\\HP_HARRY.csv', '.\\data\\\\test\\HP_HERMIONE.csv', '.\\data\\\\test\\HP_RON.csv']
stemming 시작
['shouldv', 'known', 'would', 'professor', 'mcgonagal', 'im', 'afraid', 'professor', 'good', 'bad', 'hagrid', 'bring', 'ah', 'professor', 'would', 'trust', 'hagrid', 'life', 'problem', 'trust', 'hagrid', 'albu', 'realli', 'think', 'safe', 'leav', 'peopl', 'famili', 'exactli', 'he', 'far', 'better', 'grow', 'away', 'he', 'readi', 'hagrid', 'realli', 'goodby', 'good', 'luckharri', 'potter', 'startofterm', 'notic', 'wish', 'announc', 'first', 'year', 'pleas', 'note', 'dark', 'forest', 'strictli', 'forbidden', 'student', 'also', 'caretak', 'mr', 'filch', 'ask', 'remind', 'thirdfloor', 'corridor', 'right', 'hand', 'side', 'bound', 'everyon', 'wish', 'die', 'pain', 'death', 'thank', 'let', 'feast', 'begin', 'sillllleeennnnceeee', 'everyon', 'pleas', 'panic', 'prefect', 'lead', 'hous', 'back', 'dormitori', 'tea

In [22]:
'''HP data Vectorization'''
# import numpy as np
# from sklearn.feature_extraction.text import TfidfVectorizer

hx_stemData = []
hy=[]

for element in harryPotter.keys():
    hx_stemData.append(harryPotter[element])
    hy.append(element)
    
HX_data = tfidf.transform(hx_stemData)
print(HX_data)
HY_data = np.array(hy)
print(HY_data)

'''for element in harryPotter.keys():
    tfidf.fit(harryPotter[element])
    script = tfidf.transform(harryPotter[element])
    X.append(script)
    y.append(element)

print(X.shape)'''

  (0, 38733)	0.015074029239194804
  (0, 38708)	0.027775141727860046
  (0, 38578)	0.01836618968207626
  (0, 38359)	0.035599013924146654
  (0, 38346)	0.03455395991323384
  (0, 38324)	0.05275201062479306
  (0, 38051)	0.021534953120006046
  (0, 37964)	0.05888993031465197
  (0, 37892)	0.011056175947563598
  (0, 37804)	0.033434095273694765
  (0, 37752)	0.011535924584955032
  (0, 37711)	0.03200884589074846
  (0, 37588)	0.055254951444815306
  (0, 37580)	0.04890953804469501
  (0, 37475)	0.02062841204020602
  (0, 37437)	0.020123335670527903
  (0, 37339)	0.025298583553423275
  (0, 37319)	0.01303708116998017
  (0, 37306)	0.019747343507305358
  (0, 37273)	0.030783160520346315
  (0, 36966)	0.07122735039393112
  (0, 36931)	0.04738032427553982
  (0, 36837)	0.11615732468422096
  (0, 36748)	0.028003274345576996
  (0, 36624)	0.02576865728289446
  :	:
  (4, 2826)	0.024664081766815914
  (4, 2788)	0.028905960949153802
  (4, 2780)	0.05560360077103729
  (4, 2649)	0.00923887707074336
  (4, 2456)	0.017328446181

'for element in harryPotter.keys():\n    tfidf.fit(harryPotter[element])\n    script = tfidf.transform(harryPotter[element])\n    X.append(script)\n    y.append(element)\n\nprint(X.shape)'

In [23]:
import joblib  
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from matplotlib import pyplot

model = joblib.load('XGBoost_undersample.pkl')
# xgb_model_loaded.get_xgb_params()    # Get xgboost specific parameters.
harrypotter_predict = model.predict(HX_data)
print(harrypotter_predict)

['ESTJ' 'ESTJ' 'INTP' 'ESFJ' 'ESFP']
